In [1]:
!pip install arxiv requests gradio openai pathlib2 -q

import os
import requests
import arxiv
import json
import time
import re
from datetime import datetime
from typing import List, Dict, Any
import gradio as gr
from pathlib import Path
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import logging
import subprocess
import sys
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

@dataclass
class Paper:
    title: str
    authors: List[str]
    abstract: str
    url: str
    pdf_url: str = ""
    venue: str = ""
    year: int = 0
    citations: int = 0
    doi: str = ""
    keywords: List[str] = None

class KaggleConfig:
    BASE_NOTES_PATH = Path("/kaggle/working/research_notes")
    TEMP_PATH = Path("/kaggle/working/temp")
    MAX_PAPERS_PER_SOURCE = 25
    MAX_CONCURRENT_REQUESTS = 3
    ARXIV_DELAY = 2
    SEMANTIC_SCHOLAR_DELAY = 2
    DEFAULT_SUMMARY_TYPE = "comprehensive"
    MAX_SUMMARY_LENGTH = 400

    @classmethod
    def setup_directories(cls):
        cls.BASE_NOTES_PATH.mkdir(parents=True, exist_ok=True)
        cls.TEMP_PATH.mkdir(parents=True, exist_ok=True)
        print(f"📁 Created directories:")
        print(f"   - Research notes: {cls.BASE_NOTES_PATH}")
        print(f"   - Temp files: {cls.TEMP_PATH}")

# Setup directories
KaggleConfig.setup_directories()

class AcademicPaperFetcher:
    def __init__(self):
        self.arxiv_client = arxiv.Client()
        self.semantic_scholar_base_url = "https://api.semanticscholar.org/graph/v1"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36'
        })
    
    def fetch_from_arxiv(self, query: str, max_results: int = 10) -> List[Paper]:
        try:
            print(f"🔍 Searching ArXiv for: {query}")
            
            search = arxiv.Search(
                query=query,
                max_results=min(max_results, 15),
                sort_by=arxiv.SortCriterion.Relevance
            )
            
            papers = []
            for i, result in enumerate(self.arxiv_client.results(search)):
                if i >= max_results:
                    break
                    
                paper = Paper(
                    title=result.title.strip(),
                    authors=[author.name for author in result.authors],
                    abstract=result.summary.strip(),
                    url=result.entry_id,
                    pdf_url=result.pdf_url,
                    year=result.published.year if result.published else 0
                )
                papers.append(paper)
                
                time.sleep(KaggleConfig.ARXIV_DELAY)
            
            print(f"✅ Fetched {len(papers)} papers from ArXiv")
            return papers
            
        except Exception as e:
            print(f"❌ Error fetching from ArXiv: {e}")
            return []
    
    def fetch_from_semantic_scholar(self, query: str, limit: int = 10) -> List[Paper]:
        try:
            print(f"🔍 Searching Semantic Scholar for: {query}")
            
            url = f"{self.semantic_scholar_base_url}/paper/search"
            params = {
                "query": query,
                "limit": min(limit, 15),
                "fields": "title,abstract,venue,externalIds,fieldsOfStudy,year,authors,citationCount,url"
            }
            
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json().get("data", [])
            
            papers = []
            for item in data:
                if not item.get("abstract") or len(item.get("abstract", "")) < 50:
                    continue
                    
                authors = [author.get("name", "") for author in item.get("authors", [])]
                paper = Paper(
                    title=item.get("title", "No title").strip(),
                    authors=authors,
                    abstract=item.get("abstract", "").strip(),
                    url=item.get("url", ""),
                    venue=item.get("venue", ""),
                    year=item.get("year", 0),
                    citations=item.get("citationCount", 0),
                    doi=item.get("externalIds", {}).get("DOI", ""),
                    keywords=item.get("fieldsOfStudy", [])
                )
                papers.append(paper)
            
            print(f"✅ Fetched {len(papers)} papers from Semantic Scholar")
            time.sleep(KaggleConfig.SEMANTIC_SCHOLAR_DELAY)
            return papers
            
        except Exception as e:
            print(f"❌ Error fetching from Semantic Scholar: {e}")
            return []

class PaperSummarizer:
    def __init__(self, api_key: str = None):
        self.api_key = api_key
        self.mock_mode = not api_key
        
        if api_key:
            try:
                # Try to import openai and set up
                import openai
                openai.api_key = api_key
                self.openai = openai
            except ImportError:
                print("⚠️ OpenAI package not available, using mock summaries")
                self.mock_mode = True
    
    def summarize_paper(self, paper: Paper, summary_type: str = "comprehensive") -> Dict[str, str]:
        if self.mock_mode:
            return self._generate_mock_summary(paper, summary_type)
        
        try:
            prompts = {
                "brief": f"""
                Provide a brief 2-3 sentence summary of this research paper:
                
                Title: {paper.title}
                Abstract: {paper.abstract[:500]}
                """,
                
                "comprehensive": f"""
                Provide a comprehensive summary of this research paper including:
                1. Main research question/problem
                2. Methodology used  
                3. Key findings
                4. Significance and implications
                
                Title: {paper.title}
                Authors: {', '.join(paper.authors[:3])}
                Abstract: {paper.abstract[:800]}
                """,
                
                "technical": f"""
                Provide a technical summary focusing on:
                1. Technical approach and methods
                2. Experimental setup
                3. Results and metrics
                4. Limitations and future work
                
                Title: {paper.title}
                Abstract: {paper.abstract[:800]}
                """
            }
            
            prompt = prompts.get(summary_type, prompts["comprehensive"])
            
            # Updated for newer OpenAI API
            response = self.openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an expert academic researcher who specializes in summarizing research papers clearly and concisely."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=KaggleConfig.MAX_SUMMARY_LENGTH,
                temperature=0.3
            )
            
            summary = response.choices[0].message.content.strip()
            
            return {
                "summary": summary,
                "summary_type": summary_type,
                "generated_at": datetime.now().isoformat(),
                "method": "openai"
            }
            
        except Exception as e:
            print(f"⚠️ OpenAI summarization failed: {e}")
            return self._generate_mock_summary(paper, summary_type)
    
    def _generate_mock_summary(self, paper: Paper, summary_type: str) -> Dict[str, str]:
        abstract = paper.abstract
        
        if summary_type == "brief":
            sentences = abstract.split('. ')
            summary = '. '.join(sentences[:2]) + '.'
        elif summary_type == "comprehensive":
            summary = f"""
            **Research Problem:** This paper addresses {abstract.split('.')[0].lower()}.
            
            **Methodology:** The authors employ research methods described in their abstract to investigate the problem.
            
            **Key Findings:** Based on the abstract, the main contributions appear to focus on {paper.title.lower()}.
            
            **Significance:** This work contributes to the field by advancing understanding of the research topic.
            
            **Note:** This is a structured summary based on the paper's abstract. For detailed analysis, please review the full paper.
            """
        else:  # technical
            summary = f"""
            **Technical Approach:** The methodology involves techniques related to {paper.title.lower()}.
            
            **Experimental Details:** Specific experimental setup details would be found in the full paper.
            
            **Results:** The abstract suggests findings related to the research objectives.
            
            **Limitations:** Full limitations would be detailed in the complete paper.
            
            **Note:** This is a technical overview based on available abstract information.
            """
        
        return {
            "summary": summary.strip(),
            "summary_type": summary_type,
            "generated_at": datetime.now().isoformat(),
            "method": "mock"
        }
    
    def batch_summarize(self, papers: List[Paper], summary_type: str = "comprehensive") -> Dict[str, Dict]:
        summaries = {}
        
        print(f"📝 Generating {summary_type} summaries for {len(papers)} papers...")
        
        max_workers = min(2, len(papers))
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_paper = {
                executor.submit(self.summarize_paper, paper, summary_type): paper 
                for paper in papers
            }
            
            for i, future in enumerate(future_to_paper):
                paper = future_to_paper[future]
                try:
                    summary_data = future.result()
                    summaries[paper.title] = summary_data
                    print(f"   ✅ Summarized paper {i+1}/{len(papers)}")
                except Exception as e:
                    print(f"   ❌ Failed to summarize: {paper.title[:50]}...")
                    summaries[paper.title] = {
                        "summary": "Summary generation failed", 
                        "error": str(e),
                        "summary_type": summary_type
                    }
        
        return summaries

class NotesOrganizer:
    def __init__(self, base_path: str = None):
        self.base_path = Path(base_path) if base_path else KaggleConfig.BASE_NOTES_PATH
        self.base_path.mkdir(parents=True, exist_ok=True)
    
    def create_folder_structure(self, topic: str) -> Path:
        clean_topic = re.sub(r'[^\w\s-]', '', topic).strip()
        clean_topic = re.sub(r'[-\s]+', '_', clean_topic)[:50]
        
        topic_folder = self.base_path / clean_topic
        topic_folder.mkdir(exist_ok=True)
        
        subfolders = ["papers", "summaries", "notes", "references"]
        for subfolder in subfolders:
            (topic_folder / subfolder).mkdir(exist_ok=True)
        
        print(f"📁 Created folder structure: {topic_folder}")
        return topic_folder
    
    def save_paper_data(self, paper: Paper, topic_folder: Path) -> str:
        try:
            papers_folder = topic_folder / "papers"
            
            clean_title = re.sub(r'[^\w\s-]', '', paper.title)[:50]
            clean_title = re.sub(r'[-\s]+', '_', clean_title)
            
            paper_file = papers_folder / f"{clean_title}.json"
            
            paper_data = {
                "title": paper.title,
                "authors": paper.authors,
                "abstract": paper.abstract,
                "url": paper.url,
                "pdf_url": paper.pdf_url,
                "venue": paper.venue,
                "year": paper.year,
                "citations": paper.citations,
                "doi": paper.doi,
                "keywords": paper.keywords,
                "saved_at": datetime.now().isoformat()
            }
            
            with open(paper_file, 'w', encoding='utf-8') as f:
                json.dump(paper_data, f, indent=2, ensure_ascii=False)
            
            return str(paper_file)
            
        except Exception as e:
            print(f"❌ Error saving paper data: {e}")
            return ""
    
    def save_summary(self, paper_title: str, summary_data: Dict, topic_folder: Path) -> str:
        try:
            summaries_folder = topic_folder / "summaries"
            
            clean_title = re.sub(r'[^\w\s-]', '', paper_title)[:50]
            clean_title = re.sub(r'[-\s]+', '_', clean_title)
            
            summary_file = summaries_folder / f"{clean_title}_summary.txt"
            
            with open(summary_file, 'w', encoding='utf-8') as f:
                f.write(f"Paper: {paper_title}\n")
                f.write("=" * 50 + "\n\n")
                f.write(f"Summary Type: {summary_data.get('summary_type', 'N/A')}\n")
                f.write(f"Generated: {summary_data.get('generated_at', 'N/A')}\n")
                f.write(f"Method: {summary_data.get('method', 'N/A')}\n\n")
                f.write("Summary:\n")
                f.write(summary_data.get('summary', 'No summary available'))
                f.write("\n\n")
            
            return str(summary_file)
            
        except Exception as e:
            print(f"❌ Error saving summary: {e}")
            return ""
    
    def create_research_report(self, topic: str, papers: List[Paper], summaries: Dict, topic_folder: Path) -> str:
        try:
            report_file = topic_folder / f"{topic.replace(' ', '_')}_research_report.md"
            
            with open(report_file, 'w', encoding='utf-8') as f:
                f.write(f"# Research Report: {topic}\n\n")
                f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write(f"Generated in: Kaggle Notebook Environment\n\n")
                
                f.write(f"## Summary\n\n")
                f.write(f"This report contains **{len(papers)} papers** related to '{topic}'.\n\n")
                
                years = [p.year for p in papers if p.year > 0]
                if years:
                    f.write(f"**Year Range:** {min(years)} - {max(years)}\n")
                
                total_citations = sum(p.citations for p in papers if p.citations > 0)
                if total_citations > 0:
                    f.write(f"**Total Citations:** {total_citations}\n")
                
                f.write("\n## Papers and Summaries\n\n")
                
                for i, paper in enumerate(papers, 1):
                    f.write(f"### {i}. {paper.title}\n\n")
                    f.write(f"**Authors:** {', '.join(paper.authors[:5])}")
                    if len(paper.authors) > 5:
                        f.write(f" *and {len(paper.authors)-5} others*")
                    f.write("\n\n")
                    
                    if paper.year:
                        f.write(f"**Year:** {paper.year}  ")
                    if paper.venue:
                        f.write(f"**Venue:** {paper.venue}  ")
                    if paper.citations:
                        f.write(f"**Citations:** {paper.citations}")
                    f.write("\n\n")
                    
                    f.write(f"**Abstract:** {paper.abstract}\n\n")
                    
                    summary_data = summaries.get(paper.title, {})
                    if summary_data.get('summary'):
                        f.write(f"**AI Summary ({summary_data.get('summary_type', 'N/A')}):** \n")
                        f.write(f"{summary_data['summary']}\n\n")
                    
                    f.write(f"**URL:** [{paper.url}]({paper.url})\n\n")
                    if paper.pdf_url:
                        f.write(f"**PDF:** [{paper.pdf_url}]({paper.pdf_url})\n\n")
                    
                    f.write("---\n\n")
                
                f.write("## Generated by Personal Research Assistant\n")
                f.write("*Powered by ArXiv and Semantic Scholar APIs*\n")
            
            print(f"📊 Research report created: {report_file}")
            return str(report_file)
            
        except Exception as e:
            print(f"❌ Error creating research report: {e}")
            return ""

class PersonalResearchAssistant:
    def __init__(self, openai_api_key: str = None):
        self.fetcher = AcademicPaperFetcher()
        self.summarizer = PaperSummarizer(openai_api_key)
        self.organizer = NotesOrganizer()
       
    def research_topic(self, 
                      topic: str, 
                      max_papers: int = 10, 
                      sources: List[str] = ["arxiv", "semantic_scholar"],
                      summary_type: str = "comprehensive") -> Dict[str, Any]:
        
        try:
            print(f"\n🔬 Starting research on topic: '{topic}'")
            print(f"📊 Parameters: max_papers={max_papers}, sources={sources}, summary_type={summary_type}")
            
            topic_folder = self.organizer.create_folder_structure(topic)
            
            all_papers = []
            
            if "arxiv" in sources:
                arxiv_papers = self.fetcher.fetch_from_arxiv(topic, max_papers // 2)
                all_papers.extend(arxiv_papers)
            
            if "semantic_scholar" in sources:
                scholar_papers = self.fetcher.fetch_from_semantic_scholar(topic, max_papers // 2)
                all_papers.extend(scholar_papers)
            
            unique_papers = self._remove_duplicate_papers(all_papers)
            unique_papers = unique_papers[:max_papers]
            
            print(f"📚 Found {len(unique_papers)} unique papers after deduplication")
            
            if not unique_papers:
                return {
                    "topic": topic,
                    "papers_found": 0,
                    "status": "error",
                    "error": "No papers found for the given topic"
                }
            
            summaries = self.summarizer.batch_summarize(unique_papers, summary_type)
            
            saved_files = []
            print(f"💾 Saving {len(unique_papers)} papers and summaries...")
            
            for i, paper in enumerate(unique_papers):
                paper_file = self.organizer.save_paper_data(paper, topic_folder)
                if paper_file:
                    saved_files.append(paper_file)
                
                summary_data = summaries.get(paper.title, {})
                if summary_data.get('summary'):
                    summary_file = self.organizer.save_summary(paper.title, summary_data, topic_folder)
                    if summary_file:
                        saved_files.append(summary_file)
                
                print(f"   ✅ Saved paper {i+1}/{len(unique_papers)}")
            
            report_file = self.organizer.create_research_report(topic, unique_papers, summaries, topic_folder)
            if report_file:
                saved_files.append(report_file)
            
            result = {
                "topic": topic,
                "papers_found": len(unique_papers),
                "folder_path": str(topic_folder),
                "files_created": saved_files,
                "papers": unique_papers,
                "summaries": summaries,
                "report_file": report_file,
                "status": "success"
            }
            
            print(f"\n🎉 Research completed successfully!")
            print(f"📁 Files saved to: {topic_folder}")
            print(f"📄 Created {len(saved_files)} files")
            
            return result
            
        except Exception as e:
            error_msg = str(e)
            print(f"❌ Error during research: {error_msg}")
            return {
                "topic": topic,
                "papers_found": 0,
                "status": "error",
                "error": error_msg
            }
    
    def _remove_duplicate_papers(self, papers: List[Paper]) -> List[Paper]:
        unique_papers = []
        seen_titles = set()
        
        for paper in papers:
            title_key = re.sub(r'[^\w\s]', '', paper.title.lower())[:60]
            if title_key not in seen_titles:
                seen_titles.add(title_key)
                unique_papers.append(paper)
        
        return unique_papers

def create_kaggle_gradio_interface(assistant: PersonalResearchAssistant):
    def research_interface(topic, max_papers, sources, summary_type, api_key):
        if api_key and api_key.strip():
            assistant.summarizer.api_key = api_key.strip()
            assistant.summarizer.mock_mode = False
            try:
                import openai
                openai.api_key = api_key.strip()
                assistant.summarizer.openai = openai
                print("🔑 API key updated!")
            except ImportError:
                print("⚠️ OpenAI package not available")
        
        source_list = []
        if isinstance(sources, list):
            if "ArXiv" in sources:
                source_list.append("arxiv")
            if "Semantic Scholar" in sources:
                source_list.append("semantic_scholar")
        
        if not source_list:
            return "❌ Please select at least one source.", "", ""
        
        if not topic.strip():
            return "❌ Please enter a research topic.", "", ""
        
        if len(topic.strip()) < 3:
            return "❌ Please enter a more specific research topic (at least 3 characters).", "", ""
        
        try:
            result = assistant.research_topic(
                topic=topic.strip(),
                max_papers=max_papers,
                sources=source_list,
                summary_type=summary_type
            )
            
            if result["status"] == "error":
                return f"❌ Error: {result['error']}", "", ""
            
            summary_text = f"""✅ **Research Completed Successfully!**

**Topic:** {result['topic']}
**Papers Found:** {result['papers_found']}
**Files Created:** {len(result['files_created'])}
**Folder Path:** {result['folder_path']}

**Summary Statistics:**
- Total papers processed: {result['papers_found']}
- Files generated: {len(result['files_created'])}
- Research report: Generated ✅

**Created Files:**"""
            
            for file_path in result['files_created'][:10]:
                file_name = os.path.basename(file_path)
                summary_text += f"\n• {file_name}"
            
            if len(result['files_created']) > 10:
                summary_text += f"\n• ... and {len(result['files_created']) - 10} more files"
            
            papers_overview = "# 📚 Papers Found\n\n"
            
            for i, paper in enumerate(result['papers'][:8], 1):
                papers_overview += f"## {i}. {paper.title}\n\n"
                
                authors_text = ', '.join(paper.authors[:4])
                if len(paper.authors) > 4:
                    authors_text += f" *and {len(paper.authors)-4} others*"
                papers_overview += f"**Authors:** {authors_text}\n\n"
                
                metadata = []
                if paper.year:
                    metadata.append(f"Year: {paper.year}")
                if paper.venue:
                    metadata.append(f"Venue: {paper.venue}")
                if paper.citations:
                    metadata.append(f"Citations: {paper.citations}")
                
                if metadata:
                    papers_overview += f"**Details:** {' | '.join(metadata)}\n\n"
                
                abstract_text = paper.abstract[:300]
                if len(paper.abstract) > 300:
                    abstract_text += "..."
                papers_overview += f"**Abstract:** {abstract_text}\n\n"
                
                summary_data = result['summaries'].get(paper.title, {})
                if summary_data.get('summary'):
                    summary_text_short = summary_data['summary'][:200]
                    if len(summary_data['summary']) > 200:
                        summary_text_short += "..."
                    papers_overview += f"**AI Summary:** {summary_text_short}\n\n"
                
                papers_overview += "---\n\n"
            
            if len(result['papers']) > 8:
                papers_overview += f"*... and {len(result['papers']) - 8} more papers in the full report*\n\n"
            
            report_content = ""
            if result.get('report_file') and os.path.exists(result['report_file']):
                try:
                    with open(result['report_file'], 'r', encoding='utf-8') as f:
                        report_content = f.read()
                        
                    if len(report_content) > 10000:
                        report_content = report_content[:10000] + "\n\n... [Report truncated for display. Full report saved to file] ..."
                        
                except Exception as e:
                    report_content = f"❌ Error reading report file: {e}"
            else:
                report_content = "❌ Report file not found"
            
            return summary_text, papers_overview, report_content
            
        except Exception as e:
            error_msg = f"❌ Unexpected error during research: {str(e)}"
            print(error_msg)
            return error_msg, "", ""
    
    interface = gr.Interface(
        fn=research_interface,
        inputs=[
            gr.Textbox(
                label="🔍 Research Topic",
                placeholder="Enter your research topic (e.g., 'machine learning interpretability', 'quantum computing', 'natural language processing')",
                lines=2,
                max_lines=3
            ),
            gr.Slider(
                minimum=5,
                maximum=25,
                value=10,
                step=5,
                label="📊 Maximum Papers",
                info="Number of papers to fetch (5-25)"
            ),
            gr.CheckboxGroup(
                choices=["ArXiv", "Semantic Scholar"],
                value=["ArXiv", "Semantic Scholar"],
                label="📚 Paper Sources",
                info="Select one or more sources"
            ),
            gr.Radio(
                choices=["brief", "comprehensive", "technical"],
                value="comprehensive",
                label="📝 Summary Type",
                info="Choose the type of AI-generated summary"
            ),
            gr.Textbox(
                label="🔑 OpenAI API Key (Optional)",
                placeholder="Enter your OpenAI API key for AI summaries (or leave blank for structured summaries)",
                type="password",
                info="Required only for OpenAI-powered summaries"
            )
        ],
        outputs=[
            gr.Textbox(
                label="📊 Research Summary", 
                lines=12,
                max_lines=15
            ),
            gr.Markdown(
                label="📚 Papers Overview",
                height=400
            ),
            gr.Textbox(
                label="📄 Full Research Report", 
                lines=25,
                max_lines=30
            )
        ],
        title="🔬 Personal Research Assistant",
        description="""
        ### Your AI-Powered Research Companion 🤖
        
        This tool helps you discover, organize, and summarize academic papers from ArXiv and Semantic Scholar.
        
        **Features:**
        - 📚 Search multiple academic databases
        - 🤖 AI-powered paper summaries (with OpenAI API key)
        - 📁 Organized file system with structured notes
        - 📊 Comprehensive research reports
        
        **Usage:**
        1. Enter your research topic
        2. Select paper sources and summary type
        3. Optionally provide OpenAI API key for enhanced summaries
        4. Click Submit to start research
        
        Files are saved to `/kaggle/working/research_notes/`
        """,
        examples=[
            ["machine learning interpretability", 10, ["ArXiv", "Semantic Scholar"], "comprehensive", ""],
            ["quantum computing algorithms", 15, ["ArXiv"], "technical", ""],
            ["transformer neural networks", 12, ["Semantic Scholar"], "brief", ""],
            ["reinforcement learning robotics", 8, ["ArXiv", "Semantic Scholar"], "comprehensive", ""]
        ],
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        .description {
            font-size: 14px;
        }
        """
    )
    
    return interface

def run_demo_research():
    print("🧪 Running demo research...")
    
    assistant = PersonalResearchAssistant()
    
    result = assistant.research_topic(
        topic="machine learning",
        max_papers=5,
        sources=["arxiv"],
        summary_type="brief"
    )
    
    if result["status"] == "success":
        print(f"✅ Demo completed! Found {result['papers_found']} papers")
        print(f"📁 Files saved to: {result['folder_path']}")
        
        print("\n📄 Created files:")
        for file_path in result['files_created']:
            print(f"   • {os.path.basename(file_path)}")
    else:
        print(f"❌ Demo failed: {result['error']}")
    
    return result

def main():
    print("🔬 Personal Research Assistant - Kaggle Edition")
    print("=" * 60)
    
    assistant = PersonalResearchAssistant()
    
    interface = create_kaggle_gradio_interface(assistant)
    
    interface.launch(
        share=True,
        debug=False,
        show_error=True,
        height=800,
        server_name="0.0.0.0",
        server_port=7860
    )
    
    return interface

# Run the main function
if __name__ == "__main__":
    main()


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.3 MB/s eta 0:00:00
📁 Created directories:
   - Research notes: /kaggle/working/research_notes
   - Temp files: /kaggle/working/temp
🔬 Personal Research Assistant - Kaggle Edition
* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://9245066085be5cfa23.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
